<a href="https://colab.research.google.com/github/RuchitaThombare/AtlantaCrime/blob/main/Atlanta_Crime_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
#import pandas_profiling as pp
import numpy as np
#data preprocessing
from sklearn.model_selection import train_test_split
#import models
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import StandardScaler
#visualisation
#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns
#EDA
from collections import Counter

class convertToDate:

  def __init__(self, raw_date):
    self.raw_date = raw_date


  def convertToDate(self):
    temp_list = []
    for x in self.raw_date:
      if x != 'nan':
        xsplit = x.split('-')
        x = xsplit[1] + '/' + xsplit[2] + '/' + xsplit[0]
      else:
        x = 0
      temp_list.append(x)
    return temp_list


class convertToTime:

  def __init__(self, raw_time):
    self.raw_time = raw_time


  def convertToTime(self):
    temp_list = []
    for x in self.raw_time:
      if x != 'nan':
        xsplit = list(x)
        if len(x) == 4:
          x = str(xsplit[0]) + str(xsplit[1]) + ':' + str(xsplit[2]) + str(xsplit[3])
        if len(x) == 3:
          x = '0'+ str(xsplit[0])+ ':' + str(xsplit[1])  + str(xsplit[2])
        if len(x) < 3:
          x = '00:00'
      else:
        x = '00:00'
      temp_list.append(x)
    return temp_list


df0 = pd.read_csv('COBRA-2009-2019.csv')
df1 = pd.read_csv('COBRA-2020-OldRMS-09292020.csv')

df0 = df0.drop(columns = ['Apartment Office Prefix','Apartment Number','Location Type','IBR Code','NPU', 'Possible Date', 'Possible Time'])
df0 = df0.rename(columns ={'Report Number': 'offense_id', 'Report Date':'rpt_date', 'Occur Date':'occur_date', 'Occur Time':'occur_time', 'Possible Date':'poss_date', 'Possible Time':'poss_time', 'Beat':'beat','Location':'location', 'Shift Occurence':'watch', 'UCR Literal':'crime_type', 'UCR #':'crime_type_number', 'Neighborhood':'neighborhood', 'Latitude':'lat', 'Longitude':'long'})

raw_rpt_date_obj = convertToDate(df0['rpt_date'])
df0['rpt_date'] = raw_rpt_date_obj.convertToDate()

raw_occur_date_obj = convertToDate(df0['occur_date'])
df0['occur_date'] = raw_occur_date_obj.convertToDate()

df0['occur_time'] = df0['occur_time'].astype(str)
raw_occur_time_obj = convertToTime(df0['occur_time'])
df0['occur_time'] = raw_occur_time_obj.convertToTime()

#df0 = df0.astype((columns ={'Report Number': int, 'Report Date':'rpt_date', 'Occur Date':'occur_date', 'Occur Time':'occur_time', 'Possible Date':'poss_date', 'Possible Time':'poss_time', 'Beat':'beat','Location':'location', 'Shift Occurence':'watch', 'UCR Literal':'crime_type', 'UCR #':'crime_type_number', 'Neighborhood':'neighborhood'}))
df0.info()
df1 = df1.drop(columns = ['apartment_office_prefix', 'apartment_number', 'location_type', 'npu', 'poss_date', 'poss_time'])
df1 = df1.rename(columns = {'UC2_Literal':'crime_type', 'UCR_Number':'crime_type_number'})
df1.info()
df = pd.concat([df0, df1])
df.info()
df


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342914 entries, 0 to 342913
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   offense_id         342914 non-null  int64  
 1   rpt_date           342914 non-null  object 
 2   occur_date         342914 non-null  object 
 3   occur_time         342914 non-null  object 
 4   beat               342890 non-null  float64
 5   location           342912 non-null  object 
 6   watch              342914 non-null  object 
 7   crime_type         342914 non-null  object 
 8   crime_type_number  342914 non-null  int64  
 9   neighborhood       330551 non-null  object 
 10  lat                342914 non-null  float64
 11  long               342914 non-null  float64
dtypes: float64(3), int64(2), object(7)
memory usage: 31.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14831 entries, 0 to 14830
Data columns (total 12 columns):
 #   Column             Non-Nul

,offense_id,rpt_date,occur_date,occur_time,beat,location,watch,crime_type,crime_type_number,neighborhood,lat,long
0,90010930,01/01/2009,01/01/2009,11:45,411.0,2841 GREENBRIAR PKWY,Day Watch,LARCENY-NON VEHICLE,630,Greenbriar,33.68845,-84.49328
1,90011083,01/01/2009,01/01/2009,13:30,511.0,12 BROAD ST SW,Day Watch,LARCENY-NON VEHICLE,630,Downtown,33.75320,-84.39201
2,90011208,01/01/2009,01/01/2009,15:00,407.0,3500 MARTIN L KING JR DR SW,Unknown,LARCENY-NON VEHICLE,630,Adamsville,33.75735,-84.50282
3,90011218,01/01/2009,01/01/2009,14:50,210.0,3393 PEACHTREE RD NE,Evening Watch,LARCENY-NON VEHICLE,630,Lenox,33.84676,-84.36212
4,90011289,01/01/2009,01/01/2009,16:00,411.0,2841 GREENBRIAR PKWY SW,Unknown,LARCENY-NON VEHICLE,630,Greenbriar,33.68677,-84.49773
...,...,...,...,...,...,...,...,...,...,...,...,...
14826,202670466,9/23/2020,9/22/2020,18:00,609.0,1207 HARDEE ST NE,Morning Watch,AUTO THEFT,730,Edgewood,33.75524,-84.34801
14827,202670717,9/23/2020,9/22/2020,21:30,609.0,195 ARIZONA AVE NE,Day Watch,AUTO THEFT,730,Edgewood,33.75929,-84.33120
14828,202680455,9/24/2020,9/23/2020,23:00,609.0,1661 LA FRANCE ST NE,Morning Watch,AUTO THEFT,730,Edgewood,33.76103,-84.33185
14829,202731007,9/29/2020,9/26/2020,18:00,607.0,694 SCHUYLER AVE SE,Evening Watch,AUTO THEFT,730,Boulevard Heights,33.72520,-84.36533
